In [1]:
# Policy_Management_System/ Folder Build up
#----------------------
# 7. notebook_app.ipynb (widget-based Entry Point)
#----------------------

# Cell 1 - Auto-reload + Imports must be the first cell)

# Auto reload modules
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets
from IPython.display import display, clear_output

# Core system imports
from policyholder import PolicyManager
from product import ProductManager
from payment import PaymentManager
from display_utils import display_accounts, send_payment_reminders
from export_utils import (
     export_all_extended,
     export_unified_csv,
     write_timestamped_readme,
     zip_policy_management_system
)


In [2]:
# Cell 2 - Initialize Managers (Persistent State)
pm = PolicyManager()
prodm = ProductManager()
paym = PaymentManager(pm, prodm)

output = widgets.Output()

In [3]:
# Cell 2b -Widget Inputs Setup

# Policyholder Widgets
holder_id = widgets.IntText(description="Holder ID:")
holder_name = widgets.Text(description="Name:")
holder_age = widgets.IntText(description="Age:")

holder_action = widgets.Dropdown(
    options=["Suspend", "Activate"],
    description="Action"

)

# Product widgets
product_id = widgets.IntText(description="Product ID:")
product_name = widgets.Text(description="Product Name:")
product_premium = widgets.FloatText(description="Premium:")

product_action = widgets.Dropdown(
    options=["Suspend", "Activate"],
    description="Action:"

)

# Payment widget
product_ids = widgets.Text(
    description="Product IDs (comma):"
)

# Output area
output = widgets.Output()





In [4]:
# Cell 3 - Action Handlers 
def safe_run(func):
    """ Graceful execution Wrapper"""
    with output:
        clear_output()
        try:
            func()
        except Exception as e:
            print(f"Error: {e}")


#-----------------
# Policyholder actions
#-------------------
def register_policyholder():
    try:
        pm.register_policyholder(
            holder_id.value,
            holder_name.value.strip(),
            holder_age.value
        )
        print("Policyholder registered")
    except Exception as e:
        print(f"{e}")

def suspend_activate_policyholder():
    if holder_action.value == "Suspend":
        pm.suspend_policyholder(holder_id.value)
        print("Policyholder suspended")
    else:
        pm.activate_policyholder(holder_id.value)
        print("Policyholder activated")

#---------------
# Product actions
#----------------
def add_product():
    try:
        # Ensure product id is int, name is stripped string, premium is float
        product_id_int = int(str(product_id.value).strip())
        product_name_str = str(product_name.value).strip()
        product_premium_float = float(str(product_premium.value).strip())
        
        prodm.create_product(
            product_id_int,
            product_name_str,
            product_premium_float
        )
        print("Product added successfully")
    except Exception as e:
        print(f" Error: {e}")
    

def suspend_activate_product():
    try:
        product_obj = prodm.get_product(product_id.value)
        
        if product_action.value == "Suspend":
            product_obj.suspend()
            print("Product suspended")
        else:
            product_obj.activate()
            print("Product activate")
    except Exception as e:
        print(f"Error: {e}")
        
    
#---------------
# Payment Actions
#---------------
def create_invoice():
    paym.create_payment_from_string(
        holder_id.value,
        product_ids.value
    )
    print("invoices(s) created")
    

def apply_penalties():
    paym.apply_penalties()
    print("Penalties applied")
    

def reminders():
    send_payment_reminders(paym)


#-----------------
# Display & Export
# -----------------
# Ensure penalties are applied before display
def show_accounts():
    paym.apply_penalties()
    display_accounts(pm, prodm, paym)
    
# Ensure penalties are applied before export
def export_data():
    paym.apply_penalties()
    export_all_extended(pm, prodm, paym)
    print("Data exported")


def zip_system():
    zip_policy_management_system()
    print("System zipped")

      

In [5]:
# Cell 4 - Widgets - UI Components

holder_id = widgets.IntText(description="Holder ID")
holder_name = widgets.Text(description="Name")
holder_age = widgets.IntText(description="Age")

holder_action = widgets.Dropdown(
    options=["Suspend", "Activate"],
    description="Action"
)

# Product Inputs
product_ids = widgets.Text(description="Product IDs (comma separated)")
product_name = widgets.Text(description="Product Name")
product_premium = widgets.FloatText(description="Premium")

product_action = widgets.Dropdown(
    options=["Suspend", "Activate"],
    description="Action"
)

# Product IDs for creating invoices 
product_ids = widgets.Text(description="Product IDs (comma-separated)")

# For making payments
payment_ids_to_pay = widgets.Text(
    description="Product IDs (comma-separated)",
    placeholder="e.g., 201,202"
)

# Output table for payments
payment_table_output = widgets.Output()

In [6]:
# Cell 5 - Creare Buttons (Menu Replacement)
buttons = [
    widgets.Button(description="Register Policyholder"),
    widgets.Button(description="Suspend/Activate Policyholder"),
    widgets.Button(description="Add Product"),
    widgets.Button(description="Suspend/Activate Product"),
    widgets.Button(description="Create Payment Invoice"),
    widgets.Button(description="Make Payment"),
    widgets.Button(description="Apply Penalties"),
    widgets.Button(description="Send Payment Reminders"),
    widgets.Button(description="Display Accounts"),
    widgets.Button(description="Export Data"),
    widgets.Button(description="Zip System"),
    widgets.Button(description="Refresh Invoices Table")
]

    

In [7]:
# Cell 6 - Wire Buttons to Actions
#---------------------
# Policyholder action
#---------------------
buttons[0].on_click(lambda b: safe_run(register_policyholder))
buttons[1].on_click(lambda b: safe_run(suspend_activate_policyholder))

#-------------------------
# Product Action
#-------------------------
buttons[2].on_click(lambda b: safe_run(add_product))
buttons[3].on_click(lambda b: safe_run(suspend_activate_product))

#-------------------------------------
# Payment Action
#--------------------------------------

# Create Unpaid Invoices (does not mark payment)
#--------------------------
def create_payment_invoice_ui(b):
    """
    Trigger "Create Payment Invoice" button."""
    
    with output:
        try:          
            # convert holder ID to Int
            holder_id_str = str(holder_id.value).strip()
            holder_id_val = int(holder_id_str)

            #Ensure product_ids is string and strip spaces
            product_ids_val = str(product_ids.value).strip()
            
            # Use PaymentManager wrapper to parse and create
            created_payments = paym.create_payments_from_string(holder_id_val, product_ids_val)
       
            # Apply penalties immediately so the UI shows the correct state
            paym.apply_penalties()
        
            print(f"{len(created_payments)} Invoice(s) processed and penalties updated")
   
        except Exception as e:
            print(f"Error: {e}")

   
# Attach to button 4

buttons[4].on_click(create_payment_invoice_ui)

# Make payment (marks invoices as paid)
#---------------------
def make_payment_ui(b): 
    """Trigger "Make Payment" button."""
    with output:
        try:
            # Get payment IDs to mark as paid
            ids_str = payment_ids_to_pay.value.strip()
            if not ids_str:
                raise ValueError("Please enter Payment IDs to pay")
            
            # Conver comma-separated IDs to integers
            payment_ids = [int(pid.strip()) for pid in ids_str.split(",")]

            for pid in payment_ids:
                payment = paym.get_payment(pid)
                if not payment:
                    print(f"(payment ID {pid} not found)")
                    continue
                payment.mark_paid()

            # Update penalties dynamically
            paym.apply_penalties()
            
            print(f" Selected Payment(s) marked as paid: {payment_ids}")

        except Exception as e:
            print(f" Error: {e}")

# Attach to button 5                
buttons[5].on_click(make_payment_ui)   
         
# Existing callbacks for other buttons
buttons[6].on_click(lambda b: safe_run(apply_penalties))
buttons[7].on_click(lambda b: safe_run(reminders))

# update Display Accounts to apply penalties before showing
def display_accounts_with_penalty(b):
    paym.apply_penalties()   # Ensure penalties & overdue status are current
    safe_run(show_accounts)

buttons[8].on_click(display_accounts_with_penalty)

# Updated export Data to apply penalties before export
def export_data_with_penalty(b):
    paym.apply_penalties()  # Ensure penalities & overdue status are current
    safe_run(export_data)

buttons[9].on_click(export_data_with_penalty)

# Zip System button
buttons[10].on_click(lambda b: safe_run(zip_system))

#-------------------
# Refresh Payment Table
#-------------------
def refresh_payment_table(_=None):
    with payment_table_output:
        payment_table_output.clear_output()
        from IPython.display import HTML, display
        
        headers = ["payment ID", "Holder ID", "Product ID", "Amount", "Paid", "Overdue", "Penalty"]
        rows = []
       
        for p in paym.payments:
            d = p.to_dict()
            rows.append([
                d["payment_id"],
                d["holder_id"],
                d["product_id"],
                d["amount"],
                "Yes" if d["paid"] else "No",
                "Yes" if d["overdue"] else "No",
                d["penalty"]
            ])
       
        # Build HTML table with proper styles for alignment
        html = """
        <table style="border-collapse: collapse; width: 100%; text-align: center;">
        <thead>
            <tr style="background-color: "#f2f2f2;">
        """
        html +="".join([f"<th style='border: 1px solid black; padding: 4px;'>{h}</th>" for h in headers]) 
        html += "</tr></thead><tbody>"
        
        for r in rows:
            html += "<tr>" + "".join([f"<td style='border: 1px solid black; padding: 4px;'>{c}</td>" for c in r]) + "</tr>"
            
        
        html += "</tbody></table>"

        # wrap HTML string with HTML() for proper redering
        display(HTML(html))
buttons[11].on_click(refresh_payment_table)

        

In [8]:
# Cell 7 - Display Dashboard (Widget Layout)
display(
    widgets.VBox([
        widgets.HTML("<h2> Policy Management System</h2>"),
        
        # Policyholder inputs
        holder_id, holder_name, holder_age, holder_action,
        
        # Product inputs
        product_id, product_name, product_premium, product_action,
        
        # product IDS for creating invoices
        product_ids,

        # Payment IDs for making payment
        payment_ids_to_pay,
        
        # Buttons (callbacks alrady update in cell 6)
        widgets.VBox(buttons),

        # Dynamic payment table
        payment_table_output,
        
        # Output display area
        output
    ])
)